In [114]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Flatten, Dropout
import cv2
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pylab as plt
import pickle
from keras.models import Model
from keras.applications import MobileNet, EfficientNetB0

In [20]:
training_data = []

In [117]:
classes = ['Ford' , 'Honday' , 'Hyundai' , 'Nissan' ,'Renault' , 'Suzuki' , 'Tata' , 'Toyota' , 'Volkswagen']

In [116]:
dir = r'C:\Projects\e-AUTO\eAuto_photos\eAuto_photos\photos'

In [23]:
for category in classes:
    path = os.path.join(dir , category)
    label = classes.index(category)
    
    

    for img in os.listdir(path):
        
        
        try:
            image = cv2.imread(os.path.join(path , img))
            if image is not None:
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                img_resize = cv2.resize(image_rgb , (224,224))
                
            else:
                print(f"Error loading image: {os.path.join(path, img)}")
        except Exception as e:
            print(f"Error processing image: {os.path.join(path, img)}")
            print(f"Error message: {str(e)}")
            continue
        
        training_data.append([img_resize, label])


In [24]:
import random

random.shuffle(training_data)

In [25]:
len(training_data)

6460

In [26]:
x = []
y = []

In [27]:
for features , label in training_data:
    x.append(features)
    y.append(label)

In [28]:
X = np.array(x)
Y = np.array(y)

In [29]:
X = X/255.0

In [41]:
pickle_out = open('x.pickle' , 'wb')
pickle.dump(X , pickle_out)
pickle_out.close()

pickle_out = open('y.pickle' , 'wb')
pickle.dump(Y , pickle_out)
pickle_out.close()

MemoryError: 




Model Summary:

This code implements a transfer learning approach using the MobileNet model for image classification. MobileNet is a pre-trained deep learning model that has been trained on the ImageNet dataset, which contains a wide range of images from various categories.

In the code, the MobileNet model is loaded with pre-trained weights using the MobileNet(weights='imagenet') function. The model's top layer (fully connected layer) is excluded by setting include_top=False, as we intend to add your our final layer for classification. The input shape is specified as (224, 224, 3) to match the expected input dimensions of the MobileNet model.

To fine-tune the model, the code freezes the first 15 layers of the base MobileNet model by setting their trainable attribute to False. This means that during training, only the remaining layers after the 15th layer will be updated with new weights.

The data is preprocessed using an ImageDataGenerator instance, which performs various augmentation techniques such as rescaling the pixel values. The augmented data is then divided into training and validation subsets using the validation_split argument.

The model is compiled with the sparse categorical cross-entropy loss function (loss='sparse_categorical_crossentropy') and the Adam optimizer (optimizer='adam'). The chosen metrics for evaluation are accuracy (metrics=['accuracy']).

#1

In [118]:
base_model = MobileNet(weights='imagenet' , include_top=False , input_shape=(224,224,3))

In [96]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [151]:
for layer in base_model.layers[1:15]:
    layer.trainable = False

for layer in base_model.layers[15:]:
    layer.trainable=True

In [120]:
from keras.regularizers import l2

In [156]:
model = keras.Sequential([])
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256 , activation='relu' , kernel_regularizer = l2(0.01)))
# model.add(Dropout(0.3))
model.add(Dense(128 , activation='relu' , kernel_regularizer = l2(0.01)))
# model.add(Dropout(0.2))
model.add(Dense(64 , activation='relu' , kernel_regularizer=l2(0.01)))


# model.add(keras.layers.Dense(128 , activation='relu'))
model.add(keras.layers.Dense(9 , activation='softmax'))

In [150]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 50176)             0         
                                                                 
 dense_33 (Dense)            (None, 256)               12845312  
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_34 (Dense)            (None, 128)               32896     
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                     

In [122]:
from keras.optimizers import Adam

In [123]:
check_point = ModelCheckpoint(
    filepath = 'best_model.1',
    monitor = 'val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=1
)

In [124]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    mode='max',
    verbose=1
)

In [132]:
datagen = ImageDataGenerator(
    
    rescale=1/255.0,
    validation_split=0.2
    # rotation_range=10,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=[0.9,1.2],
    # horizontal_flip=False,
    # fill_mode = 'reflect',
    # brightness_range=(0.8,1.2)
)

In [189]:
train_generator = datagen.flow_from_directory(
    dir,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse',
    subset = 'training'


)

Found 6179 images belonging to 9 classes.


In [44]:
print(train_generator.class_indices)

{'Ford': 0, 'Honday': 1, 'Hyundai': 2, 'Nissan': 3, 'Renault': 4, 'Suzuki': 5, 'Tata': 6, 'Toyota': 7, 'Volkswagen': 8}


In [190]:
validation_generator = datagen.flow_from_directory(
    dir,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse',
    subset='validation'


)

Found 1541 images belonging to 9 classes.


#1

In [54]:
# regularization_strength = 0.01
# for layer in updated_model.layers:
#     if isinstance(layer, Dense):
#         layer.kernel_regularizer = keras.regularizers.l2(regularization_strength)

In [157]:
model.compile(loss = 'sparse_categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])

#2

In [158]:
model.fit(
    train_generator,
    epochs=10,
    validation_data = validation_generator,
    callbacks=[early_stopping,check_point]
    
)

Epoch 1/10
  6/194 [..............................] - ETA: 7:50 - loss: 5.1719 - accuracy: 0.8438

c:\Users\karth\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


194/194 [==============================] - ETA: 0s - loss: 1.2475 - accuracy: 0.9743
Epoch 1: val_accuracy did not improve from 0.94549
194/194 [==============================] - 544s 3s/step - loss: 1.2475 - accuracy: 0.9743 - val_loss: 0.6635 - val_accuracy: 0.9429
Epoch 2/10
194/194 [==============================] - ETA: 0s - loss: 0.5686 - accuracy: 0.9717
Epoch 2: val_accuracy did not improve from 0.94549
194/194 [==============================] - 453s 2s/step - loss: 0.5686 - accuracy: 0.9717 - val_loss: 1.8481 - val_accuracy: 0.7339
Epoch 3/10
194/194 [==============================] - ETA: 0s - loss: 0.4039 - accuracy: 0.9751
Epoch 3: val_accuracy did not improve from 0.94549
194/194 [==============================] - 503s 3s/step - loss: 0.4039 - accuracy: 0.9751 - val_loss: 0.3538 - val_accuracy: 0.9429
Epoch 4/10
194/194 [==============================] - ETA: 0s - loss: 0.1739 - accuracy: 0.9943
Epoch 4: val_accuracy improved from 0.94549 to 0.98183, saving model to best_m

INFO:tensorflow:Assets written to: best_model.1\assets


INFO:tensorflow:Assets written to: best_model.1\assets


194/194 [==============================] - 545s 3s/step - loss: 0.1739 - accuracy: 0.9943 - val_loss: 0.1841 - val_accuracy: 0.9818
Epoch 5/10


c:\Users\karth\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


194/194 [==============================] - ETA: 0s - loss: 0.1695 - accuracy: 0.9934
Epoch 5: val_accuracy did not improve from 0.98183
194/194 [==============================] - 459s 2s/step - loss: 0.1695 - accuracy: 0.9934 - val_loss: 0.1943 - val_accuracy: 0.9695
Epoch 6/10
194/194 [==============================] - ETA: 0s - loss: 0.1359 - accuracy: 0.9929
Epoch 6: val_accuracy did not improve from 0.98183
194/194 [==============================] - 473s 2s/step - loss: 0.1359 - accuracy: 0.9929 - val_loss: 0.4414 - val_accuracy: 0.9358
Epoch 7/10
194/194 [==============================] - ETA: 0s - loss: 0.2495 - accuracy: 0.9861
Epoch 7: val_accuracy did not improve from 0.98183
194/194 [==============================] - 432s 2s/step - loss: 0.2495 - accuracy: 0.9861 - val_loss: 0.2459 - val_accuracy: 0.9676
Epoch 7: early stopping


Model Name : best_model.1

Evaluation report :

Epoch 1:

    Loss: 1.2475
    Accuracy: 0.9743
    Validation Loss: 0.6635
    Validation Accuracy: 0.9429

Epoch 2:

    Loss: 0.5686
    Accuracy: 0.9717
    Validation Loss: 1.8481
    Validation Accuracy: 0.7339

Epoch 3:

    Loss: 0.4039
    Accuracy: 0.9751
    Validation Loss: 0.3538
    Validation Accuracy: 0.9429

Epoch 4:

    Loss: 0.1739
    Accuracy: 0.9943
    Validation Loss: 0.1841
    Validation Accuracy: 0.9818

Epoch 5:

    Loss: 0.1695
    Accuracy: 0.9934
    Validation Loss: 0.1943
    Validation Accuracy: 0.9695

Epoch 6:

    Loss: 0.1359
    Accuracy: 0.9929
    Validation Loss: 0.4414
    Validation Accuracy: 0.9358
    
Epoch 7:

    Loss: 0.2495
    Accuracy: 0.9861
    Validation Loss: 0.2459
    Validation Accuracy: 0.9676


So, this model achieved a maximum validation accuracy of 98 percent and this is considered to be the best model of all the other models I trained.

*I would chose to deploy this model in production because:

    Performance : 
A validation accuracy of 98% indicates that my model is able to make accurate predictions on unseen data. This level of accuracy suggests that my model has learned meaningful patterns and can effectively classify the car images based on their logos.

    Efficient Architecture: 
MobileNet+(my_fully_connected_layers) is designed to be a lightweight architecture suitable for mobile and resource-constrained environments. It strikes a balance between model size and accuracy, making it computationally efficient and faster to deploy compared to larger models with similar performance.



In [304]:
img = cv2.imread(r'C:\Projects\e-AUTO\suzuki.jpg')
img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
img = cv2.resize(img , (224,224))
img = img/255
img = np.expand_dims(img , axis=0)

predictions = model.predict(img)

output = np.argmax(predictions[0])
class_ = classes[output]
print(class_)



1/1 [==============================] - 0s 114ms/step
Suzuki


Top-2 Model

This model too is based on the MobileNet architecture, which is a lightweight convolutional neural network (CNN) designed for efficient image classification tasks. The MobileNet model consists of depthwise separable convolutions, which significantly reduce the number of parameters while maintaining good accuracy. In the code, the pre-trained MobileNet model with weights from the ImageNet dataset is used as the base model. The first 15 layers of the base model are frozen, meaning they are not trainable, while the remaining layers are made trainable to adapt the model to the specific task at hand.


The "model2" is a custom model built on top of the base MobileNet model. It starts with the base model's output and adds additional layers to perform further feature extraction and classification. In the code, a Flatten layer is added to convert the 4D output of the base model into a 2D feature vector. This is followed by three Dense layers with ReLU activation, which serve as fully connected layers for learning complex patterns in the features. Each Dense layer has kernel regularization (L2) applied to control overfitting. The final Dense layer has 9 units with softmax activation, representing the probabilities of the input belonging to each class in the dataset.

In [289]:
base_model = MobileNet(weights='imagenet' , include_top=False , input_shape=(224,224,3))

In [ ]:
for layer in base_model.layers[1:15]:
    layer.trainable = False

for layer in base_model.layers[15:]:
    layer.trainable=True

In [292]:
model2 = keras.Sequential([])
model2.add(base_model)
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(256 , activation='relu' , kernel_regularizer = l2(0.01)))
# model.add(Dropout(0.3))
model2.add(Dense(128 , activation='relu' , kernel_regularizer = l2(0.01)))
# model.add(Dropout(0.2))
model2.add(Dense(64 , activation='relu' , kernel_regularizer=l2(0.01)))


# model.add(keras.layers.Dense(128 , activation='relu'))
model2.add(Dense(9 , activation='softmax'))

In [293]:
train_generator = datagen.flow_from_directory(
    dir,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse',
    subset = 'training'


)

Found 6179 images belonging to 9 classes.


In [294]:
validation_generator = datagen.flow_from_directory(
    dir,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse',
    subset='validation'


)

Found 1541 images belonging to 9 classes.


In [295]:
check_point = ModelCheckpoint(
    filepath = 'best_model.2',
    monitor = 'val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=1
)

In [296]:
model2.compile(loss = 'sparse_categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])

In [299]:
model2.fit(
    train_generator,
    epochs=7,
    validation_data = validation_generator,
    callbacks=[early_stopping,check_point],
    batch_size=64
    
)

Epoch 1/7
 10/194 [>.............................] - ETA: 8:33 - loss: 0.3609 - accuracy: 0.9812

c:\Users\karth\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


194/194 [==============================] - ETA: 0s - loss: 0.6032 - accuracy: 0.9547
Epoch 1: val_accuracy did not improve from 0.97469
194/194 [==============================] - 583s 3s/step - loss: 0.6032 - accuracy: 0.9547 - val_loss: 0.6109 - val_accuracy: 0.9156
Epoch 2/7
194/194 [==============================] - ETA: 0s - loss: 0.5464 - accuracy: 0.9621 
Epoch 2: val_accuracy did not improve from 0.97469
194/194 [==============================] - 6327s 33s/step - loss: 0.5464 - accuracy: 0.9621 - val_loss: 0.5735 - val_accuracy: 0.9215
Epoch 3/7
194/194 [==============================] - ETA: 0s - loss: 0.3436 - accuracy: 0.9804
Epoch 3: val_accuracy did not improve from 0.97469
194/194 [==============================] - 575s 3s/step - loss: 0.3436 - accuracy: 0.9804 - val_loss: 0.6730 - val_accuracy: 0.9150
Epoch 4/7
194/194 [==============================] - ETA: 0s - loss: 0.4299 - accuracy: 0.9717
Epoch 4: val_accuracy did not improve from 0.97469
194/194 [==================

Model name : mobilenet_extension.2

So, this model achieved a maximum validation accuracy of 97 percent and this is considered to be the second best model of all the other models I trained.

*I would also suggest to chose to deploy this model in production because:

    Performance : 
A validation accuracy of 97% indicates that my model is able to make accurate predictions on unseen data. This model is also equally performing well on unseen data.

    Efficient Architecture: 
MobileNet+(my_fully_connected_layers) is designed to be a lightweight architecture suitable for mobile and resource-constrained environments. It strikes a balance between model size and accuracy, making it computationally efficient and faster to deploy compared to larger models with similar performance.



Top-3 Model

This model too is based on the MobileNet architecture, which is a lightweight convolutional neural network (CNN) designed for efficient image classification tasks. The MobileNet model consists of depthwise separable convolutions, which significantly reduce the number of parameters while maintaining good accuracy. In the code, the pre-trained MobileNet model with weights from the ImageNet dataset is used as the base model. The first 15 layers of the base model are frozen, meaning they are not trainable, while the remaining layers are made trainable to adapt the model to the specific task at hand.


The "model3" is a custom model built on top of the base MobileNet model. It starts with the base model's output and adds additional layers to perform further feature extraction and classification. In the code, a Flatten layer is added to convert the 4D output of the base model into a 2D feature vector. This is followed by three Dense layers,128 128 and 64 units respectively with ReLU activation, which serve as fully connected layers for learning complex patterns in the features. Each Dense layer has kernel regularization (L2) applied to control overfitting. The final Dense layer has 9 units with softmax activation, representing the probabilities of the input belonging to each class in the dataset.

In [308]:
base_model = MobileNet(weights='imagenet' , include_top=False , input_shape=(224,224,3))

In [309]:
for layer in base_model.layers[1:10]:
    layer.trainable = False

for layer in base_model.layers[10:]:
    layer.trainable=True

In [310]:
train_generator = datagen.flow_from_directory(
    dir,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse',
    subset = 'training'


)

Found 6179 images belonging to 9 classes.


In [311]:
validation_generator = datagen.flow_from_directory(
    dir,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse',
    subset='validation'


)

Found 1541 images belonging to 9 classes.


In [312]:
check_point = ModelCheckpoint(
    filepath = 'best_model.3',
    monitor = 'val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=1
)

In [313]:
model3 = keras.Sequential([])
model3.add(base_model)
model3.add(keras.layers.Flatten())
model3.add(keras.layers.Dense(128 , activation='relu' , kernel_regularizer = l2(0.01)))
# model.add(Dropout(0.3))
model3.add(Dense(128 , activation='relu' , kernel_regularizer = l2(0.01)))
# model.add(Dropout(0.2))
model3.add(Dense(64 , activation='relu' , kernel_regularizer=l2(0.01)))
model3.add(Dense(32 , activation = 'relu' , kernel_regularizer=l2(0.01)))



model3.add(Dense(9 , activation='softmax'))

In [314]:
model3.compile(loss = 'sparse_categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])

In [315]:
model3.fit(
    train_generator,
    epochs=10,
    validation_data = validation_generator,
    callbacks=[early_stopping,check_point],
    batch_size=64
    
)

Epoch 1/10
 21/194 [==>...........................] - ETA: 7:12 - loss: 8.3958 - accuracy: 0.1577

c:\Users\karth\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


194/194 [==============================] - ETA: 0s - loss: 4.1818 - accuracy: 0.5370
Epoch 1: val_accuracy improved from -inf to 0.29851, saving model to best_model.3


INFO:tensorflow:Assets written to: best_model.3\assets


INFO:tensorflow:Assets written to: best_model.3\assets


...

INFO:tensorflow:Assets written to: best_model.3\assets


INFO:tensorflow:Assets written to: best_model.3\assets


194/194 [==============================] - 398s 2s/step - loss: 0.5727 - accuracy: 0.9639 - val_loss: 0.6489 - val_accuracy: 0.9241
Epoch 8/10
  2/194 [..............................] - ETA: 5:44 - loss: 0.4479 - accuracy: 0.9688

c:\Users\karth\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


194/194 [==============================] - ETA: 0s - loss: 0.4106 - accuracy: 0.9851
Epoch 8: val_accuracy did not improve from 0.92408
194/194 [==============================] - 397s 2s/step - loss: 0.4106 - accuracy: 0.9851 - val_loss: 1.5606 - val_accuracy: 0.7962
Epoch 9/10
194/194 [==============================] - ETA: 0s - loss: 0.5833 - accuracy: 0.9689
Epoch 9: val_accuracy did not improve from 0.92408
194/194 [==============================] - 359s 2s/step - loss: 0.5833 - accuracy: 0.9689 - val_loss: 0.5917 - val_accuracy: 0.9228
Epoch 10/10
194/194 [==============================] - ETA: 0s - loss: 0.3598 - accuracy: 0.9814
Epoch 10: val_accuracy did not improve from 0.92408
194/194 [==============================] - 375s 2s/step - loss: 0.3598 - accuracy: 0.9814 - val_loss: 0.6809 - val_accuracy: 0.8968
Epoch 10: early stopping


Model name : mobilenet_extension.3

So, this model achieved a maximum validation accuracy of 92 percent and this is considered to be the 3rd best model of all the other models I trained.

Evaluation metrics : val_accuracy

    

